In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
from functools import partial
import itertools
import os
from scipy.integrate import solve_ivp
import pandas as pd
import traceback
import copy
import multiprocessing as mp
import sys

from mpl_toolkits.axes_grid1.inset_locator import InsetPosition
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)


%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams.update({'font.size': 13})

In [ ]:
def sci_notation(number, sig_fig=2):
    form_string = "{0:.{1:d}e}".format(number, sig_fig)
    a,b = form_string.split("e")
    b = int(b)
    c = r"$10^{"+str(b)+r"}$"
    if float(number).is_integer() and number <= 10:
        return str(int(number))
    elif a == '1.00':
        return c
    elif float(a).is_integer():
        return str(int(float(a))) + r"$\cdot$" + c
    elif b == 0:
        return a
    else:
        return a + r"$\cdot$" + c

In [ ]:
def axes_tick_formatter(self, MajorTicks, MinorTicks, axis = 'x'):
    if axis == 'x':
        self.xaxis.set_major_locator(MultipleLocator(MajorTicks))
        self.xaxis.set_minor_locator(MultipleLocator(MinorTicks))
    elif axis == 'y':
        self.yaxis.set_major_locator(MultipleLocator(MajorTicks))
        self.yaxis.set_minor_locator(MultipleLocator(MinorTicks))
    else:
        print("Algo mal hay")

In [ ]:
def remove_zeros(a):
    
    if a[-1] != '0' and a[-1] != '.':
        return str(a)   
        
    else:
        c = str(a[:-1])
        return(remove_zeros(c))

def param_folder_notation(number, sig_fig=2):
    if number == 0.0:
        c = '0'
    
    else:    
        form_string = "{0:.{1:d}e}".format(number, sig_fig)
        a,b = form_string.split("e")
        a = remove_zeros(a)
        b = int(b)

        if float(a).is_integer():
            if b == 0:
                c = str(int(float(a)))
            else:
                c = str(int(float(a))) +'e'+ str(b)
        else:
            if b == 0:
                c = a
            else:
                c = a +'e'+ str(b)
        c = c.replace('-','m').replace('.','p')
    
    return c

In [ ]:
def m_notch_epsin_coop(t, x, params):
    bl = params["bl"]
    dl = params["dl"]
    
    kr = params["kr"]
    krl = params["krl"]
    
    br = params["br"]
    dr = params["dr"]
    
    kb = params["kb"]
    ku = params["ku"]
    
    km = params["km"]
    n = params["n"]
    m = params["m"]
    
    
    ks2 = params["ks2"]
    kp = params["kp"]
    
    ki = params["ki"]
    ks3 = params["ks3"]
    ds = params["ds"]
    
    KG = params["KG"]
    TN = params["TN"]
    CN = params["CN"]
    
    dG = params["dG"]
    dm = params["dm"]
    
    R = x[0]
    L = x[1]
    E = x[2]
    C = x[3]
    CL = x[4]
    CLi = x[5]
    CR = x[6]
    S = x[7]
    G4 = x[8]
    Cit = x[9]

    
    Dx = np.array([
        br - dr * R - kb * (R * L) + ku * C, #R
        bl - dl * L - kb * (R * L) + ku * C + n * krl * kr * CLi , #L
        m * kr * CLi - m * ks2 * km ** (1-m) * C ** n * E ** m , #E
#         0., #E
        kb * R * L - ku * C - kp * C - n * ks2 * km ** (1-m) * C ** n * E ** m, #C
        ks2 * km ** (1-m) * C ** n * E ** m - ki * CL, #CL
        ki * CL - kr * CLi, #CLi
        ks2 * km ** (1-m) * C ** n * E ** m - ks3 * CR, #CR
        n * ks3 * CR - ds * S, #S
        n * ks3 * CR - dG * G4, #G4
        CN * TN * G4 / (G4 + KG) - dm * Cit #Cit
        
    ])
    
    return Dx

In [ ]:
def is_zero(a):
    return(1 if int(a) == int(0) else 0)
is_zero_vec = np.vectorize(is_zero)

def x_interpolation(r, q, y):
    if r[0] >= q[0]:
        p_0 = q
        p_1 = r
    else:
        p_0 = r
        p_1 = q

    x = p_0[0] + (y - p_0[1]) / (p_1[1] - p_0[1]) * (p_1[0] - p_0[0])

    return x

def peak_values (np_input_x, np_input_y, eps = 0.1):
    max_value = max(np_input_y)
    max_value_list = np.where(np_input_y == max_value)
    max_value_index = int(np.round(np.mean(max_value_list[0])))

    end_value = np_input_y[-1]

    t2 = 0
    t1 = 0

    if end_value < 0.5 * max_value:

        half_max = 0.5 * max_value

        mvi_left = max_value_index - 1
        mvi_right = max_value_index + 1

        while np_input_y[mvi_left] > half_max:
            mvi_left -= 1

        while np_input_y[mvi_right] > half_max:
            mvi_right += 1

        r1 = [np_input_x[mvi_left], np_input_y[mvi_left]]
        q1 = [np_input_x[mvi_left + 1], np_input_y[mvi_left + 1]]

        t1 = x_interpolation(r1, q1, half_max)

        r2 = [np_input_x[mvi_right], np_input_y[mvi_right]]
        q2 = [np_input_x[mvi_right - 1], np_input_y[mvi_right - 1]]

        t2 = x_interpolation(r2, q2, half_max)
        
    max_12 = 0.5 * max_value
    
    k = 0
    
    while np_input_y[k] < max_12:
        k += 1
        
    r12 = [np_input_x[k], np_input_y[k]]
    q12 = [np_input_x[k - 1], np_input_y[k - 1]]
    
    t12 = x_interpolation(r12, q12, max_12)
    
    t_max = np_input_x[max_value_index]

    return (t1, t2, max_value, end_value, t12, t_max)

def HS_f (a,b,eps):
    if a/b>eps:
        return 1.
    else:
        return 0.

In [ ]:
def f_RS (peak):
    max_val = peak[2]
    end_val = peak[3]
    
    H = max_val - end_val
    
    return (H/end_val)

def f_RSM (peak):
    max_val = peak[2]
    end_val = peak[3]
    
    H = max_val - end_val
    
    return (H/max_val)

def f_NRS (peak, params):
    br = params['br']
    ds = params['ds']
    
    max_val = peak[2]
    end_val = peak[3]
    
    H = max_val - end_val
    S0 = br/ds
    
    return (H/S0)

def f_DT (peak, eps):

    t1 = peak[0]
    t2 = peak[1]
    
    max_val = peak[2]
    end_val = peak[3]
    
    HS = HS_f(max_val-end_val, max_val, eps)
    
    return  (t2 - t1)

def f_RT (peak, params, eps):
    ds = params['ds']

    t1 = peak[0]
    t2 = peak[1]
    
    max_val = peak[2]
    end_val = peak[3]
    
    HS = HS_f(max_val-end_val, max_val, eps)
    
    return (HS * (t2 - t1) * ds)

def f_GT(peak, mu, sd):
    t1 = peak[0]
    t2 = peak[1]
    
    dt = t2-t1
    
    return np.exp(-((mu-dt)/sd)**2)

def f_GS(rs, mu, sd):
    return np.exp(-((mu-rs)/sd)**2)

def f_GT12(peak, mu, sd):
    t12 = peak[4]
    
    return np.exp(-((mu-t12)/sd)**2)

In [ ]:
def solve_resolution(KI, KJ, PPNAMES, PPVALUES, X0, PARAMS, variable, p_gauss, model = m_notch_epsin_coop, param_iter = [],
                    time_l = [0,25*60]):
    k_i = KI
    k_j = KJ
    p_p_names = PPNAMES
    p_p_values = PPVALUES
    x_0 = X0
    params = PARAMS
    
    p_p_values_iter = list(itertools.product(*p_p_values))
    
    if param_iter == []:
        param_iter = []
        param_iter_dic = []
        for  e in p_p_values_iter:
            param_i = dict(params)
            param_i.update(zip(p_p_names, e))
            param_iter.append(tuple([dict(param_i)]))
            param_iter_dic.append(dict(param_i))
            
    else:
        param_iter_dic = []
        for  e in param_iter:
            param_iter_dic.append(dict(e[0]))
            
    if type(x_0[0])!=list:
        partial_iter = partial(par_solve_ivp, model, time_l, x_0,'Radau')
        sol = [partial_iter(e) for e in param_iter]
        
    else:
        partial_iter = partial(par_solve_ivp, model, time_l)
        sol = [partial_iter(f,'Radau',e) for e,f in zip(param_iter,x_0)]
    
    T_H = [e.t / 60. for e in sol]
    
    res = [peak_values(T_H[e], sol[e].y[variable]) for e in range(len(sol))]
    s_max = [max(sol[e].y[variable]) for e in range(len(sol))]
    s_stat = [sol[e].y[variable][-1] for e in range(len(sol))]
    table_params = [[p_p_values_iter[j] for j in range(i*len(k_j),(i+1)*len(k_j))]for i in range(len(k_i))]
    table_params_dict = [[param_iter_dic[j] for j in range(i*len(k_j),(i+1)*len(k_j))]for i in range(len(k_i))]
    table_values =  [[res[j] for j in range(i*len(k_j),(i+1)*len(k_j))]for i in range(len(k_i))]
    SMax =  [[s_max[j] for j in range(i*len(k_j),(i+1)*len(k_j))]for i in range(len(k_i))]
    Sstat =  [[s_stat[j] for j in range(i*len(k_j),(i+1)*len(k_j))]for i in range(len(k_i))]
    

    len_i = len(table_values)
    len_j = len(table_values[0])
    
    mut = p_gauss["mut"]
    sdt = p_gauss["sdt"]
    
    mut12 = p_gauss["mut12"]
    sdt12 = p_gauss["sdt12"]

    mus = p_gauss["mus"]
    sds = p_gauss["sds"]

    eps = p_gauss["eps"]

    RSM = [[f_RSM(tvij) for tvij in tvi] for tvi in table_values]

    RT = [[f_RT(table_values[i][j], table_params_dict[i][j], eps) for j in
          range(len_j)] for i in range(len_i)]
    
    RM = [[RT[i][j] * RSM[i][j] for j in range(len_j)] for i in range(len_i)]

    GSM = [[f_GS(RSM[i][j], mus, sds) for j in range(len_j)] for i in
          range(len_i)]

    GT = [[f_GT(tvij, mut, sdt) for tvij in tvi] for tvi in table_values]

    GM = [[GT[i][j] * GSM[i][j] for j in range(len_j)] for i in range(len_i)]
    
    RT12 = [[tvij[4] for tvij in tvi] for tvi in table_values]
    
    GT12 = [[f_GT12(tvij, mut12, sdt12) for tvij in tvi] for tvi in table_values]
    
    DT = [[f_DT(tvij, eps) for tvij in tvi] for tvi in table_values]
    
    TMAX = [[tvij[5] for tvij in tvi] for tvi in table_values]
    
    return [RSM, RT, RM, GSM, GT, GM, RT12, GT12, SMax, Sstat, DT, TMAX]

In [ ]:
def par_solve_ivp(model, time_l, x_0, meth, arg):
    return solve_ivp(model, time_l, x_0, method = meth, args=arg)

In [ ]:

llss = ["-", ":", "-.", "--"]

variables = ["R","L","E","C",r"$C_L$",r"$C_{Li}$",r"$C_R$","S","Gal4","Cit"]
variables_f = ["R","L","E","C",r"C_L",r"C_Li",r"C_R","S","Gal4","Cit"]
ligands = ["Dll1", r"$Dll4_{ECD}$"+"\n"+r"$Dll1_{ICD}$", "Dll4", r"$Dll1_{ECD}$"+"\n"+r"$Dll4_{ICD}$"]
ligands_f = ["Dll1", "1I4E", "Dll4", "4I1E"]
cm_variables = ["#194569","#800020","#ff7f0e","#9467bd","#c20e35","#fa8fad","#1f77b4","#80bce6","#c8f1c8","#77dd77"]
cm_ligands = ["#194569","#ff7f0e","#800020","#9467bd"]
color_bars = [cm_variables[5],cm_variables[4],cm_variables[1]]


p_gauss_S = {"mut": 0.8, "sdt": 0.2, "mut12": 0.1, "sdt12":0.02, "mus": 0.9, "sds": 0.15, "eps": 0.1}
p_gauss_Citrine = {"mut": 9.5, "sdt": 4.5, "mut12": 1.5, "sdt12":0.5, "mus": 0.8, "sds": 0.15, "eps": 0.1}

range_vals = [[0.,1.], 1, [0.,1.], [0.,1.], [0.,1.], [0.,1.], 1, [0.,1.], 0, 0, 0]


In [ ]:
t12_G = 3.9
t12_m = 3.4

params_base = {"bl": 2e0, 
    "br": 6e0, 
    "dl": 2e-3, 
    "dr": 2e-3,
    "kb": 2e-4,
    "ku": 2e2, 
    "kp": 5.,
    "ks2": 1.,
    "kr": 1e-4,
    'krl':0.,
    'km':1e3,
    "ki": 1.5e1, 
    "ks3": 1e-1, 
    "ds": 8e-3,
    "n":1,
    "m":1,
    "KG": 6.6 * 0.1 * 1200., 
    "t12_G": t12_G,
    "t12_m": t12_m,
    "dG": np.log(2) / (60. * t12_G),
    "dm": np.log(2) / (60. * t12_m),
    "E0": 7e2,
    "TN": 1/60.,
    "CN": 500.} 

t0 = 0.
tf = 25. * 60.
time_l = [t0,tf]

c_max_11 = 0.68
c_max_41 = 0.59

c_max_44 = 1.4
c_max_14 = 3.6


In [ ]:
filename0 = 'Fig2'
dir = os.getcwd() + '/' + filename0
if os.path.isdir(dir) == False:
    os.makedirs(dir)
os.chdir(dir)


# Panels A, B & E

In [ ]:
params_transient = dict(params_base)
params_sustained = dict(params_base)
params_sustained.update({"kr": 2e-3})


In [ ]:
params = params_transient

x_0 = [round(params["br"]/params["dr"],2), round(params["bl"]/params["dl"],2), 
       round(params["E0"],2), 0., 0., 0., 0., 0., 0., 0.]

partial_iter = partial(par_solve_ivp, m_notch_epsin_coop, time_l, x_0,'Radau')
sol_trans = partial_iter([(params)])
T_H_trans = [e/60. for e in sol_trans.t]


In [ ]:
params = params_sustained

x_0 = [round(params["br"]/params["dr"],2), round(params["bl"]/params["dl"],2), 
       round(params["E0"],2), 0., 0., 0., 0., 0., 0., 0.]

partial_iter = partial(par_solve_ivp, m_notch_epsin_coop, time_l, x_0,'Radau')
sol_sust = partial_iter([(params)])
T_H_sust = [e/60. for e in sol_sust.t]


## Panel A

In [ ]:

fig, ax = plt.subplots()
ax.plot(T_H_sust, sol_sust.y[9],c = cm_variables[1], label= 'Sustained (S1)')
ax.plot(T_H_trans, sol_trans.y[9],c = cm_variables[0], label= 'Transient (T1)')

ax.set_ylabel("Cit [a.u.]", fontsize=15)
ax.set_ylabel("Cit [a.u.]", fontsize=15)

ax.legend()

ax.set_xlabel("t [h]", fontsize=15)

axes_tick_formatter(ax, 5, 2.5)

axes_tick_formatter(ax, 200, 100, axis = 'y')

fig.subplots_adjust(hspace=0)

fig.savefig(filename0 + '_A.png', bbox_inches="tight", dpi = 300, transparent = True)


## Panel E

In [ ]:

fig, ax = plt.subplots()
ax.plot(T_H_sust, sol_sust.y[7],c = cm_variables[1], label= 'Sustained (S1)')
ax.plot(T_H_trans, sol_trans.y[7],c = cm_variables[0], label= 'Transient (T1)')

ax.set_ylabel("S [a.u.]", fontsize=15)
ax.set_ylabel("S [a.u.]", fontsize=15)


ax.legend()

ax.set_xlabel("t [h]", fontsize=15)

axes_tick_formatter(ax, 5, 2.5)

axes_tick_formatter(ax, 200, 100, axis = 'y')
axes_tick_formatter(ax, 150, 75, axis = 'y')

fig.subplots_adjust(hspace=0)

fig.savefig(filename0 + '_E.png', bbox_inches="tight", dpi = 300, transparent = True)


## Panel B

In [ ]:

pvs = peak_values(T_H_trans, sol_trans.y[9])

h = pvs[2] - pvs[3]

fig1, ax1 = plt.subplots()

ax1.plot(T_H_trans, sol_trans.y[9],c = cm_variables[9])

ax1.set_ylabel("Cit [a.u.]", fontsize=15)
ax1.set_xlabel("t [h]", fontsize=15)

ax1.set_ylim([-10,610])


ax1.arrow(pvs[0]*1.18, pvs[2]/2,(pvs[1]-pvs[0])*.945,0, head_width=10, head_length=0.4, color='#0B6623')
ax1.arrow(pvs[1]*0.97, pvs[2]/2,(pvs[0]-pvs[1])*.93,0, head_width=10, head_length=0.4, color='#0B6623')
ax1.text(0.22* (pvs[1]-pvs[0]),1.05 * pvs[2]/2,r"FWHM $\approx$" + str(round(pvs[1]-pvs[0], 2)) + "h", color='#0B6623')


ax1.arrow(pvs[5],10,0,pvs[2]*0.945, head_width=0.3, head_length=20, color='#DD571C')
ax1.arrow(pvs[5],pvs[2],0,-pvs[2]*0.95, head_width=0.3, head_length=20, color='#DD571C')
ax1.text(pvs[5] * 1.05,1.05 * pvs[2]/4,r"Cit$_{Max}\approx$" + str(round(pvs[2], 2)) + ' [a.u.]', color='#DD571C')

ax1.hlines(pvs[2],pvs[5], 25, ls = '--', color='#194569')
ax1.arrow(25.0,pvs[3],0,h*0.945, head_width=0.3, head_length=20, color='#194569')
ax1.arrow(25.0,pvs[2],0,-h*0.95, head_width=0.3, head_length=20, color='#194569')
ax1.text(20.5,1.25 * pvs[2]/2,r"$\rho\approx$" + str(round(h/pvs[2], 2)), color='#194569')

axes_tick_formatter(ax1, 150, 75, axis = 'y')
axes_tick_formatter(ax1, 5, 2.5)

fig1.tight_layout()

fig1.savefig(filename0 + '_B.png', bbox_inches="tight", dpi = 300, transparent = True )


# Panels C & D

In [ ]:
params = dict(params_base)

k_i = np.logspace(-5,-2,50)
k_j = np.logspace(0,4,50)

p_p_names = ["kr","E0"]
p_p_values = [k_i, k_j]

p_p_values_iter = list(itertools.product(*p_p_values))

p_p_values_iter.sort(key=lambda a: a[0])

XLabel = r"$k_r\ [min^{-1}]$"
YLabel = r"$E_0$ [a.u.]"
save=1

param_iter = []

for  e in p_p_values_iter:
    param_i = dict(params)
    param_i.update(zip(p_p_names, e))
    param_iter.append(tuple([dict(param_i)]))


x_0 = [[round(params["br"]/params["dr"],2), 
       round(params["bl"]/params["dl"],2),
       round(e[1],2),
       0., 0., 0., 0., 0., 0., 0.] for e in p_p_values_iter]


In [ ]:

maps_Citrine = solve_resolution(k_i, k_j, p_p_names, p_p_values, x_0, params, 9, p_gauss_Citrine)


In [ ]:
fig, ax = plt.subplots()

cm = copy.copy(mpl.cm.get_cmap("Blues"))
cm.set_bad(color= '#d3d3d3')

xmin = min(k_i)
ymin = min(k_j)
xmax = max(k_i)+1
ymax = max(k_j)+1

dx = 1
dy = 1

X, Y = np.meshgrid(k_i,k_j, indexing = 'ij')

Z = maps_Citrine[0]

VMIN = round(min([min([Z[i][j] for j in range(len(k_j)) if isinstance(Z[0][j], float) ]) for i in range(len(k_i))]),1)
VMAX = round(max([max([Z[i][j] for j in range(len(k_j)) if isinstance(Z[0][j], float) ]) for i in range(len(k_i))]),1)

TICKS =  np.around(np.linspace(VMIN,VMAX, 5),2)

c = ax.pcolormesh(X, Y ,Z, cmap=cm, vmin = VMIN, vmax = VMAX)

CS =ax.contour(X, Y, Z, levels = [.7,.8], colors=["#36454F","#808080"])

cbar = fig.colorbar(c, ticks = TICKS)
cbar.set_label(r"$\rho$", fontsize = 13)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlabel(XLabel, fontsize = 13)
ax.set_ylabel(YLabel, fontsize = 13)

ax.text(1e-4,7.7e2,r'$T_1$',color='#DC582A')
ax.scatter(1e-4,7e2, marker = '+',color='#DC582A')
ax.text(1e-4,1.1e2,r'$T_2$',color='#DC582A')
ax.scatter(1e-4,1e2, marker = '+',color='#DC582A', zorder = 3)
ax.text(5e-5,1.2e3,r'$T_3$',color='#DC582A')
ax.scatter(5e-5,1.1e3, marker = '+',color='#DC582A', zorder = 3)
ax.text(5e-5,1.1e2,r'$T_4$',color='#DC582A')
ax.scatter(5e-5,1e2, marker = '+',color='#DC582A', zorder = 3)


ax.text(2.1e-3,7.7e2,r'$S_1$',color='#77dd77')
ax.scatter(2e-3,7e2, marker = '+',color='#77dd77')
ax.text(2.1e-3,1.1e2,r'$S_2$',color='#77dd77')
ax.scatter(2e-3,1e2, marker = '+',color='#77dd77', zorder = 3)
ax.text(2.1e-3,5.35e3,r'$S_3$',color='#77dd77')
ax.scatter(2e-3,5e3, marker = '+',color='#77dd77', zorder = 3)
ax.text(2.1e-5,5.35e3,r'$S_4$',color='#77dd77')
ax.scatter(2e-5,5e3, marker = '+',color='#77dd77', zorder = 3)

ax.text(1.5e-5,8,'Transient', color='white')
ax.text(1e-3,2.5e3,'Sustained', color='#36454F')



ax.set_box_aspect(1)

fig.tight_layout()

fig.savefig(filename0 + '_C.png', bbox_inches="tight", dpi = 300, transparent = True)


In [ ]:
fig, ax = plt.subplots()

cm = copy.copy(mpl.cm.get_cmap("Greens"))
cm.set_bad(color= '#d3d3d3')

xmin = min(k_i)
ymin = min(k_j)
xmax = max(k_i)+1
ymax = max(k_j)+1

dx = 1
dy = 1

X, Y = np.meshgrid(k_i,k_j, indexing = 'ij')

Z = np.ma.masked_array(maps_Citrine[10], mask=is_zero_vec(maps_Citrine[10])) 

VMIN = round(min([min([Z[i][j] for j in range(len(k_j)) if isinstance(Z[0][j], float) ]) for i in range(len(k_i))]),1)
VMAX = round(max([max([Z[i][j] for j in range(len(k_j)) if isinstance(Z[0][j], float) ]) for i in range(len(k_i))]),1)

TICKS =  np.around(np.linspace(VMIN,VMAX, 5),2)

c = ax.pcolormesh(X, Y ,Z, cmap=cm, vmin = VMIN, vmax = VMAX)

CS =ax.contour(X, Y, maps_Citrine[0], levels = [.7,.8], colors=["#36454F","#808080"])

cbar = fig.colorbar(c, ticks = TICKS)
cbar.set_label(r"FWHM [h]", fontsize = 13)

# cbar.ax.set_yticklabels(TICKS)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlabel(XLabel, fontsize = 13)
ax.set_ylabel(YLabel, fontsize = 13)

ax.text(1.5e-5,8,'Transient', color='#36454F')
ax.text(1e-3,2.5e3,'Sustained', color='black')

ax.set_box_aspect(1)

fig.tight_layout()

fig.savefig(filename0 + '_D.png', bbox_inches="tight", dpi = 300, transparent = True)


# Panel S7 - C

In [ ]:
fig, ax = plt.subplots()

cm = copy.copy(mpl.cm.get_cmap("Oranges"))

cm.set_bad(color= '#d3d3d3')

xmin = min(k_i)
ymin = min(k_j)
xmax = max(k_i)+1
ymax = max(k_j)+1

dx = 1
dy = 1

X, Y = np.meshgrid(k_i,k_j, indexing = 'ij')

Z = maps_Citrine[8]

VMIN = round(min([min([Z[i][j] for j in range(len(k_j)) if isinstance(Z[0][j], float) ]) for i in range(len(k_i))]),1)
VMAX = round(max([max([Z[i][j] for j in range(len(k_j)) if isinstance(Z[0][j], float) ]) for i in range(len(k_i))]),1)

TICKS =  np.around(np.linspace(VMIN,VMAX, 5),2)

c = ax.pcolormesh(X, Y ,Z, cmap=cm, vmin = VMIN, vmax = VMAX)

CS =ax.contour(X, Y, maps_Citrine[0], levels = [.7,.8], colors=["#36454F","#808080"])

cbar = fig.colorbar(c, ticks = TICKS)
cbar.set_label(r"$Cit_{Max}$ [a.u.]", fontsize = 13)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlabel(XLabel, fontsize = 13)
ax.set_ylabel(YLabel, fontsize = 13)

ax.set_box_aspect(1)

fig.tight_layout()

fig.savefig('S7_C.png', bbox_inches="tight", dpi = 300, transparent = True)


# Panel F

In [ ]:

param_iter = []

A = 3
params_T1 = dict(params_base)
params_T1.update({"E0":7e2, "kr":1e-4})
param_iter.append(tuple([dict(params_T1)]))
params_T2 = dict(params_base)
params_T2.update({"E0":1e2, "kr":1e-4})
param_iter.append(tuple([dict(params_T2)]))
params_T3 = dict(params_base)
params_T3.update({"E0":1.1e3, "kr":5e-5})
param_iter.append(tuple([dict(params_T3)]))
params_T4 = dict(params_base)
params_T4.update({"E0":1e2, "kr":5e-5})
param_iter.append(tuple([dict(params_T4)]))

params_T1_A7 = dict(params_base)
params_T1_A7.update({"E0":7e2, "kr":1e-4, 'm':A})
param_iter.append(tuple([dict(params_T1_A7)]))


params_S1 = dict(params_base)
params_S1.update({"E0":7e2, "kr":2e-3})
param_iter.append(tuple([dict(params_S1)]))
params_S2 = dict(params_base)
params_S2.update({"E0":1e2, "kr":2e-3})
param_iter.append(tuple([dict(params_S2)]))
params_S3 = dict(params_base)
params_S3.update({"E0":5e3, "kr":2e-3})
param_iter.append(tuple([dict(params_S3)]))
params_S4 = dict(params_base)
params_S4.update({"E0":5e3, "kr":2e-5})
param_iter.append(tuple([dict(params_S4)]))

params_S1_A7 = dict(params_base)
params_S1_A7.update({"E0":7e2, "kr":2e-3, 'm':A})
param_iter.append(tuple([dict(params_S1_A7)]))

labels = ["T1", "T2","T3","T4", "T1" + "\n"+ "A = " + str(A), "S1", "S2","S3","S4", "S1" + "\n"+ "A = " + str(A) ]

x_0 = [[round(e[0]["br"]/e[0]["dr"],2), 
       round(e[0]["bl"]/e[0]["dl"],2),
       round(e[0]["E0"],2),
       0., 0., 0., 0., 0., 0., 0.] for e in param_iter]

In [ ]:
partial_iter = partial(par_solve_ivp, m_notch_epsin_coop, time_l)

sol = [partial_iter(f,'Radau',e) for e,f in zip(param_iter,x_0)]
T_H = [e.t/60. for e in sol]


In [ ]:
fig1, ax1 = plt.subplots(2,1,sharex=True)
vv = 9

for i in range(0,5):
    ls = '-'
    if i == 3 or i == 4:
        ls = '--'
    ax1[0].plot(T_H[i], sol[i].y[vv],c = cm_variables[i],ls = ls, label= labels[i])
    
for i in range(5,10):
    ls = '-'
    if i == 7 or i == 9:
        ls = '--'
    ax1[1].plot(T_H[i], sol[i].y[vv],c = cm_variables[i-5],ls = ls, label= labels[i])

ax1[0].set_ylabel(variables[vv] + ' [a.u.]')
ax1[1].set_ylabel(variables[vv] + ' [a.u.]')


ax1[0].legend(loc = [1.01,0.])
ax1[1].legend(loc = [1.01,0.])

ax1[1].set_xlabel("t [h]", fontsize=15)

axes_tick_formatter(ax1[1], 5, 2.5)

axes_tick_formatter(ax1[0], 200, 100, axis = 'y')
axes_tick_formatter(ax1[1], 300, 150, axis = 'y')


fig1.subplots_adjust(hspace=0)

fig1.savefig(filename0 + '_F'+'.png', bbox_inches="tight", dpi = 300, transparent = True)


# Panels G & I

In [ ]:
params = dict(params_base)

k_i = [1,2,4]

param_iter_T = []
list_dics_T = []

param_iter_S = []
list_dics_S = []

list_dics_T.append({'kr': 1e-4, 'E0': 7e2, 'Case': r'$T_1$'})
list_dics_T.append({'kr': 1e-4, 'E0': 1e2, 'Case': r'$T_2$'})
list_dics_T.append({'kr': 5e-5, 'E0': 1.1e3, 'Case': r'$T_3$'})
list_dics_T.append({'kr': 5e-5, 'E0': 1e2, 'Case': r'$T_4$'})

list_dics_S.append({'kr': 2e-3, 'E0': 7e2, 'Case':r'$S_1$'})
list_dics_S.append({'kr': 2e-3, 'E0': 1e2, 'Case':r'$S_2$'})
list_dics_S.append({'kr': 2e-3, 'E0': 5e3, 'Case':r'$S_3$'})
list_dics_S.append({'kr': 2e-5, 'E0': 5e3, 'Case':r'$S_4$'})

for e in list_dics_T:
    for i in k_i:
        param_i = dict(params)
        param_i.update(e)
        param_i.update({'bl':i})
        param_iter_T.append(tuple([dict(param_i)]))

for e in list_dics_S:
    for i in k_i:
        param_i = dict(params)
        param_i.update(e)
        param_i.update({'bl':i})
        param_iter_S.append(tuple([dict(param_i)]))
    
labels_T = [e[0]["Case"] for e in param_iter_T]
labels_S = [e[0]["Case"] for e in param_iter_S]


x_0_T = [[round(params_base["br"]/params_base["dr"],2), 
       round(e[0]["bl"]/params_base["dl"],2),
       round(e[0]["E0"],2),
       0., 0., 0., 0., 0., 0., 0.] for e in param_iter_T]

x_0_S = [[round(params_base["br"]/params_base["dr"],2), 
       round(e[0]["bl"]/params_base["dl"],2),
       round(e[0]["E0"],2),
       0., 0., 0., 0., 0., 0., 0.] for e in param_iter_S]


In [ ]:
partial_iter = partial(par_solve_ivp, m_notch_epsin_coop, time_l)

sol_T = [partial_iter(f,'Radau',e) for e,f in zip(param_iter_T,x_0_T)]
T_H_T = [e.t/60. for e in sol_T]

sol_S = [partial_iter(f,'Radau',e) for e,f in zip(param_iter_S,x_0_S)]
T_H_S = [e.t/60. for e in sol_S]


In [ ]:
Cit_Max_T = [max(e.y[9]) for e in sol_T]

data_T = [[param_iter_T[i][0]['bl'], param_iter_T[i][0]['E0'], param_iter_T[i][0]['kr'], param_iter_T[i][0]['Case'],Cit_Max_T[i]] for i in range(len(Cit_Max_T))]
columns_T = ['bl','E0','kr', 'Case','Cit_Max']
df_T = pd.DataFrame(data_T,columns = columns_T)

Cit_Max_S = [max(e.y[9]) for e in sol_S]

data_S = [[param_iter_S[i][0]['bl'], param_iter_S[i][0]['E0'], param_iter_S[i][0]['kr'], param_iter_S[i][0]['Case'],Cit_Max_S[i]] for i in range(len(Cit_Max_S))]
columns_S = ['bl','E0','kr', 'Case','Cit_Max']
df_S = pd.DataFrame(data_S,columns = columns_S)


In [ ]:

fig,ax = plt.subplots()

df_T.pivot(index='Case',columns='bl',values='Cit_Max').plot(kind='bar', color=color_bars, rot = 0, ax = ax)
ax.legend([r"$\beta_L = $" +str(i) for i in k_i ])
ax.set_ylabel(r'$Cit_{Max}$ [a.u]')
ax.set_xlabel(r'')
axes_tick_formatter(ax, 100, 50, axis = 'y')

fig.tight_layout()

fig.savefig(filename0 + '_G.png', bbox_inches="tight", dpi = 300, transparent = True)


In [ ]:

fig,ax = plt.subplots()

df_S.pivot(index='Case',columns='bl',values='Cit_Max').plot(kind='bar', color=color_bars, rot = 0, ax = ax)
ax.legend([r"$\beta_L = $" +str(i) for i in k_i ])
ax.set_ylabel(r'$Cit_{Max}$ [a.u]')
ax.set_xlabel(r'')
axes_tick_formatter(ax, 200, 100, axis = 'y')

fig.tight_layout()

fig.savefig(filename0 + '_I.png', bbox_inches="tight", dpi = 300, transparent = True)


# Panels H & J

In [ ]:
params = dict(params_base)
list_dics = []

list_dics.append({'kr': 1e-4, 'E0': 7e2, 'Case': r'$T_1$'})
list_dics.append({'kr': 2e-3, 'E0': 7e2, 'Case':r'$S_1$'})
list_dics.append({'kr': 2e-3, 'E0': 5e3, 'Case':r'$S_3$'})

k_i = [1,2,4]

param_iter = []
for e in list_dics:
    for i in k_i:
        param_i = dict(params)
        param_i.update(e)
        param_i.update({'bl':i})
        param_iter.append(tuple([dict(param_i)]))

x_0 = [[round(params_base["br"]/params_base["dr"],2), 
       round(e[0]["bl"]/params_base["dl"],2),
       round(e[0]["E0"],2),
       0., 0., 0., 0., 0., 0., 0.] for e in param_iter]


In [ ]:
partial_iter = partial(par_solve_ivp, m_notch_epsin_coop, time_l)

sol = [partial_iter(f,'Radau',e) for e,f in zip(param_iter,x_0)]
T_H = [e.t/60. for e in sol]

In [ ]:

fig, ax = plt.subplots()

for i in range(0,3):
    ax.plot(T_H[i], sol[i].y[9],color = color_bars[i])
    ax.plot(T_H[i], sol[i].y[7],color = color_bars[i], ls = '--' )

llss = ['-','--']
labels2 = [r"$Cit$", r"$S$"]
h2 = [plt.plot([],[], color='#161616', ls=llss[j])[0] for j in range(len(llss))]
leg2 = ax.legend(handles=h2, labels=labels2, fontsize = 12, facecolor='white', framealpha=1, loc = 1)

labels1 = [param_iter[i][0]["Case"] + r'; $\beta_L=$' + str(int(param_iter[i][0]["bl"])) for i in range(len(k_i))]
h1 = [plt.plot([],[], color=color_bars[j], ls=llss[0])[0] for j in range(len(k_i))]
leg1= ax.legend(handles=h1, labels=labels1, fontsize = 12, facecolor='white', framealpha=1, loc = (0.705,0.59))   

ax.set_ylabel("Cit, S" + ' [a.u.]')
ax.set_xlabel('t [h]')

ax.add_artist(leg1)
ax.add_artist(leg2)

axes_tick_formatter(ax, 5, 2.5)
axes_tick_formatter(ax, 150, 75, axis = 'y')

fig.savefig(filename0 + '_H.png', bbox_inches="tight", dpi = 300, transparent = True)


In [ ]:

fig, ax = plt.subplots()

for j,i in enumerate (range(6,9)):
    ax.plot(T_H[i], sol[i].y[9],color = color_bars[j])
    ax.plot(T_H[i], sol[i].y[7],color = color_bars[j], ls = '--' )

llss = ['-','--']
labels2 = [r"$Cit$", r"$S$"]
h2 = [plt.plot([],[], color='#161616', ls=llss[j])[0] for j in range(len(llss))]
leg2 = ax.legend(handles=h2, labels=labels2, fontsize = 12, facecolor='white', framealpha=1, loc = 1)

labels1 = [param_iter[i+6][0]["Case"] + r'; $\beta_L=$' + str(int(param_iter[i][0]["bl"])) for i in range(len(k_i))]
h1 = [plt.plot([],[], color=color_bars[j], ls=llss[0])[0] for j in range(len(k_i))]
leg1= ax.legend(handles=h1, labels=labels1, fontsize = 12, facecolor='white', framealpha=1, loc = (0.705,0.59))   

ax.set_ylabel("Cit, S" + ' [a.u.]')
ax.set_xlabel('t [h]')

ax.add_artist(leg1)
ax.add_artist(leg2)

axes_tick_formatter(ax, 5, 2.5)
axes_tick_formatter(ax, 300, 150, axis = 'y')

fig.savefig(filename0 + '_J.png', bbox_inches="tight", dpi = 300, transparent = True)
